# 0. Introduction

Student: Marko Kadic, 12045128
Contribution as well: Billie Postlmayrl, 01307120

The goal of this notebook is to build a good classifier, the task of which is to recognize cahracters in the Muppet Show. <br>
**Kermit the Frog** and **Waldorf & Statler** are the characters I am aiming to recognize. <br>

Furthermore, the methods used will be those of a classic ML classification, paired with **Feature Engineering** methods. <br>

The features will be of both audio and video domain. <br>

**Dependency installations:** <br>
pip install numpy <br>
pip install pandas <br>
pip install opencv-python <br>
pip install tensorflow <br>
pip install librosa <br>
pip install matplotlib <br>
pip install scipy <br>
!pip install  opencv-python==3.4.2.17 <br>
!pip install  opencv-contrib-python==3.4.2.17 <br>

**Complete Timesheet:** <br>
Installation and notebook setup: 25th November, 0.5 hours <br>
Data preprocessing: 25th November, 3 hours <br>
Watched Lecture 3/10 - Feature Engineering: 27th November, 1.5 hours <br>
Watched Lecture 7/10 -  Integral Transforms & Spectral Features: 28th November, 2 hours <br>
After watching the lectures, I have decided on which features to extract from the data: 28th November, 0.5 hours <br>
Effort to understand multiple audio signal processing methods: 4th December, 3 hours <br>
Extraction of Time Domain Audio Features: 5th December, 2 hours <br>
Extraction of Time Domain Audio Features: 6th December, 2 hours <br>
Watched Lecture 1/10 - 1.5 hrs  Jan 3rd<br>
Watched Lecture 2/10 - 1.5 hrs Jan 3d<br> 
Watched Lecture 4/10 - 1.5 hrs Jan 3rd<br> 
Watched Lecture 5/10 - 1.5 hrs Jan 3rd<br> 
Watched Lecture 9/10 - 0.5 hrs Jan 3rd<br> 
Additional Work on Time Domain Audio Features: 2 hours Jan 4th<br> 
Additional Work on Frequency Domain Audio Features: 0.5 hours Jan 4th <br> 
Setting up Google Drive Environment: 0.5 hours Jan 4th <br>
Exploration and Learning about Image Features: 5 hours Jan 4th <br> 
Extraction of Time Domain Image Features 1: 7 hours Jan 5th<br> 
Extraction of Time Domain Image Features 2: 6 hours  Jan 5th<br>
Extraction of Frequency Domain Image Features: 7 hours Jan 6th<br> 
Extraction of Local Image Features 1 (HOG): 10 hours  Jan 6th/7th<br>
Extraction of Local Image Features 2 (SIFT-bag): 8 hours  Jan <br>7th-10th
Extraction of Local Image Features 3 (Corners, SIFT): 10 hours Jan 7th-10th<br>
Feature Calculations/Calculation Time: 15 hours Jan 7th-10th <br>
Preparation of Features for Classification: 5 hours Jan 7th-10th<br>
Creation of Classifiers: 2 hours Jan 7th-10th <br>
Classification: 10 hours Jan 7th-10th <br>
Evaluation: 0.5 hours Jan 7th-10th<br>
Comments on Methods and Feature Qualities: 0.5 hours Jan 7th-10th <br>


Time Billie Postlmayrl (lab partner) 01307120: <br>
Extraction of Image features - Color Features: 5 hours <br>
Extraction of Audio features - Time Domain Audio: 6 hours <br>


In [1]:
import numpy as np
import pandas as pd
!pip install  opencv-python==3.4.2.17
!pip install  opencv-contrib-python==3.4.2.17
import cv2 
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.io import wavfile
import librosa
from skimage import feature
import scipy.spatial.distance
import scipy.cluster
import scipy
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from scipy.fftpack import dct

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/data/"

sim12-ground-truth-muppets


# 1. Preprocessing

I have extracted the audio files as separate files.
I have converted the original videos using cv2 library, into arrays of frames.

The plan is to do the audio and video feature engineering separately and later on, combine the solutions.

I have loaded up the ground truth annotations and have split them into train and test parts - 2 episodes for training, 1 episode for testing.

In [3]:
# PATHS TO FULL EPISODES
path_ep1 = "data/episodes/Muppets-02-01-01.avi"
path_ep2 = "data/episodes/Muppets-02-04-04.avi"
path_ep3 = "data/episodes/Muppets-03-04-03.avi"

# GOOGLE COLLAB ADD ON
path_ep1 = "/content/drive/MyDrive/" + path_ep1
path_ep2 = "/content/drive/MyDrive/" + path_ep2
path_ep3 = "/content/drive/MyDrive/" + path_ep3

In [ ]:
# EXTRACT AUDIO FROM EPISODES
#import subprocess

#command1 = "ffmpeg -i " + path_ep1 + " -ab 160k -ac 2 -ar 44100 -vn data/episodes/audio_ep1.wav"
#command2 = "ffmpeg -i " + path_ep2 + " -ab 160k -ac 2 -ar 44100 -vn data/episodes/audio_ep2.wav"
#command3 = "ffmpeg -i " + path_ep3 + " -ab 160k -ac 2 -ar 44100 -vn data/episodes/audio_ep3.wav"

#subprocess.call(command1, shell=True)
#subprocess.call(command2, shell=True)
#subprocess.call(command3, shell=True)

0

In [4]:
# PATHS TO AUDIO FILES
audio_ep1 = "data/episodes/audio_ep1.wav"
audio_ep2 = "data/episodes/audio_ep2.wav"
audio_ep3 = "data/episodes/audio_ep3.wav"

# GOOGLE COLLAB ADD ON
audio_ep1 = "/content/drive/MyDrive/" + audio_ep1
audio_ep2 = "/content/drive/MyDrive/" + audio_ep2
audio_ep3 = "/content/drive/MyDrive/" + audio_ep3

In [ ]:
# EXTRACT FRAMES FROM EPISODES
###################################
# Google Drive extra
gd_loc = "/content/drive/MyDrive/data/processed_data/episodes/ep1/"

# EPISODE 1
vidcap = cv2.VideoCapture(path_ep1)
(success,image) = vidcap.read()

count = 0
row_count = 0

(height, width, channels) = image.shape

frames_ep1 = []
counter = 0
batch_counter = 1
frame_batch_size = 10000
scale_percent = 50 # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)

# BATCH LOAD THE EPISODE DUE TO RAM LIMITATIONS
while success:
    if(counter >= frame_batch_size):
      frame_array_ep1 = np.array(frames_ep1)
      np.save(gd_loc + 'frame_array_ep1_'+ str(batch_counter) +'.npy', frame_array_ep1)
      del frame_array_ep1
      batch_counter += 1
      counter = 0
      frames_ep1 = []
    else:
      counter += 1

    # RESIZE IMAGE FOR NOW TO USE LESS SPACE FOR PROCESING
    try:
      resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    except:
      break
    frames_ep1.append(resized)
    (sucess,image) = vidcap.read()

frame_array_ep1 = np.array(frames_ep1)
np.save(gd_loc + 'frame_array_ep1_'+ str(batch_counter) +'.npy', frame_array_ep1)
del frame_array_ep1
del frames_ep1

###################################
# EPISODE 2
gd_loc = "/content/drive/MyDrive/data/processed_data/episodes/ep2/"

vidcap = cv2.VideoCapture(path_ep2)
(success,image) = vidcap.read()

count = 0
row_count = 0

frames_ep2 = []
counter = 0
batch_counter = 1

# BATCH LOAD THE EPISODE DUE TO RAM LIMITATIONS
while success:
    if(counter >= frame_batch_size):
      frame_array_ep2 = np.array(frames_ep2)
      np.save(gd_loc + 'frame_array_ep2_'+ str(batch_counter) +'.npy', frame_array_ep2)
      del frame_array_ep2
      batch_counter += 1
      counter = 0
      frames_ep2 = []
    else:
      counter += 1

    # RESIZE IMAGE FOR NOW TO USE LESS SPACE FOR PROCESING
    try:
      resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    except:
      break
    frames_ep2.append(resized)
    (sucess,image) = vidcap.read()

frame_array_ep2 = np.array(frames_ep2)
np.save(gd_loc + 'frame_array_ep2_'+ str(batch_counter) +'.npy', frame_array_ep2)
del frame_array_ep2
del frames_ep2


###################################    
# EPISODE 3
gd_loc = "/content/drive/MyDrive/data/processed_data/episodes/ep3/"

vidcap = cv2.VideoCapture(path_ep3)
(success,image) = vidcap.read()

count = 0
row_count = 0

frames_ep3 = []
counter = 0
batch_counter = 1

# BATCH LOAD THE EPISODE DUE TO RAM LIMITATIONS
while success:
    if(counter >= frame_batch_size):
      frame_array_ep3 = np.array(frames_ep3)
      np.save(gd_loc + 'frame_array_ep3_'+ str(batch_counter) +'.npy', frame_array_ep3)
      del frame_array_ep3
      batch_counter += 1
      counter = 0
      frames_ep3 = []
    else:
      counter += 1

    # RESIZE IMAGE FOR NOW TO USE LESS SPACE FOR PROCESING
    try:
      resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    except:
      break
    frames_ep3.append(resized)
    (sucess,image) = vidcap.read()

frame_array_ep3 = np.array(frames_ep3)
np.save(gd_loc + 'frame_array_ep3_'+ str(batch_counter) +'.npy', frame_array_ep3)
del frame_array_ep3
del frames_ep3

In [5]:
# EXTRACT TRAINING AND TESTING ANNOTTATIONS
ground_truth_ep1 = "data/sim12-ground-truth-muppets/GroundTruth_Muppets-02-01-01.csv"
ground_truth_ep2 = "data/sim12-ground-truth-muppets/GroundTruth_Muppets-02-04-04.csv"
ground_truth_ep3 = "data/sim12-ground-truth-muppets/GroundTruth_Muppets-03-04-03.csv"


# Google collab addon
ground_truth_ep1 = "/content/drive/MyDrive/" + ground_truth_ep1
ground_truth_ep2 = "/content/drive/MyDrive/" + ground_truth_ep2
ground_truth_ep3 = "/content/drive/MyDrive/" + ground_truth_ep3

annotations_train = pd.concat([pd.read_csv(ground_truth_ep1, sep=';'), pd.read_csv(ground_truth_ep2, sep=';')]).sample(frac=1)
annotations_test = pd.read_csv(ground_truth_ep3, sep=';')

kermit_train  = annotations_train['Kermit']
kermit_test = annotations_test['Kermit']

statler_waldorf_train = annotations_train['StatlerWaldorf']
statler_waldorf_test = annotations_test['StatlerWaldorf']

statler_waldorf_audio_train = annotations_train['Audio_StatlerWaldorf']
statler_waldorf_audio_test = annotations_test['Audio_StatlerWaldorf']


# 2. Lectures

After understanding the assignment I have watched 2 lecture videos on the topic of Feature Engineering.

1. Lecture 3/10 - Feature Engineering
**time / effort:** *27th November, 1.5 hours*

2. Lecture 7/10 -  Integral Transforms & Spectral Features
**time / effort:** *28th November, 2 hours*

3. Lecture 1/10 - Intoduction to Similarity Modelling

4. Lecture 2/10 - Similarity

5. Lecture 4/10 - Classification

6. Lecture 5/10 - Evaluation

After watching the lectures, I have decided on which features to extract from the data:
**time / effort:** *28th November, 0.5 hours*

**Audio:** <br>

**Time domain features:**<br>
- Loudness
- Fundamental Frequency (ZCR)
- Fundamental Frequency (YIN)
<br>

**Frequency domain features:**
- The mel frequency cepstral coefficients (MFCCs)
- Spectral peaks

**Video:** <br>
**Time domain features:** <br>
- Edge Information (Contour Information)
- Edge Orientation Histogram
- HUE Information (Dominant colour)
- HUE Information - Color Histograms :) <br>

**Frequency domain features:** <br>
- Luminance + Discrete Cosine Transform<br>

**Local methods:**<br>
- SIFT - Scale Invariant Feature Transformation (Bag of Visual Features Algorithm)
- HOG - Histogram of Oriented Gradients
- Shi-Tomasi Corner Detector


# Similarity Modelling 1


After receving the input from the proffesor the notebook has been split into Similarity Modelling 1 
and Similarity Modelling 2 parts:

The features relevant for Similarity Modelling 1:

**Audio:** <br>
Time domain features:
- Loudness
- Fundamental Frequency (ZCR)
- Fundamental Frequency (YIN)

**Video:**
<br>
Time domain features: <br>
- Edge Information (Contour Information)
- Edge Orientation Histogram
- HUE Information (Dominant colour)
- HUE Information - Color Histograms :) <br>


**A) Time domain Audio** <br>

Extraction of previously named Time Domain Audio Features:
- Loudness
- Fundamental Frequency (ZCR)
- Fundamental Frequency (YIN)

In [6]:
# LOAD UP AUDIO
# audio_ep1, audio_ep2, audio_ep3
time_audio_loc = "/content/drive/MyDrive/data/processed_data/time_audio/"

In [6]:
# LOUDNESS FEATURE CALCULATION

# Root Mean Squared value for signal frame

# frame length should change 20ms - 1s

rms_ep1_all = []
rms_ep2_all = []
rms_ep3_all = []
signal_ep1, sr  = librosa.load(audio_ep1)
for frame_length_iter in range(2048, sr, 2048):
  rms_ep1 = librosa.feature.rms(y=signal_ep1, frame_length = int(frame_length_iter))
  rms_ep1_all.append(rms_ep1)
del signal_ep1

signal_ep2, sr  = librosa.load(audio_ep2)
for frame_length_iter in range(2048, sr, 2048):
  rms_ep2 = librosa.feature.rms(y=signal_ep2, frame_length = int(frame_length_iter))
  rms_ep2_all.append(rms_ep2)
del signal_ep2

signal_ep3, sr  = librosa.load(audio_ep3)
for frame_length_iter in range(2048, sr, 2048):
  rms_ep3 = librosa.feature.rms(y=signal_ep3, frame_length = int(frame_length_iter))
  rms_ep3_all.append(rms_ep3)
del signal_ep3

np.save(time_audio_loc + 'rms_ep1.npy', rms_ep1_all)
np.save(time_audio_loc + 'rms_ep2.npy', rms_ep2_all)
np.save(time_audio_loc + 'rms_ep3.npy', rms_ep3_all)

In [8]:
# FUNDAMENTAL FREQUENCY DETECTION BASED ON THE ZERO CROSSINGS RATE
signal_ep1, sr  = librosa.load(audio_ep1)
signal_ep2, sr  = librosa.load(audio_ep2)
signal_ep3, sr  = librosa.load(audio_ep3)

zcr_ep1 = librosa.feature.zero_crossing_rate(signal_ep1)
zcr_ep2 = librosa.feature.zero_crossing_rate(signal_ep2)
zcr_ep3 = librosa.feature.zero_crossing_rate(signal_ep3)

np.save(time_audio_loc + 'zcr_ep1.npy', zcr_ep1)
np.save(time_audio_loc + 'zcr_ep2.npy', zcr_ep2)
np.save(time_audio_loc + 'zcr_ep3.npy', zcr_ep3)

In [9]:
# FUNDAMENTAL FREQUENCY DETECTION BASED ON THE YIN METHOD
# We find the fundamental frequency of the audio sample windows
signal_ep1, sr  = librosa.load(audio_ep1)
signal_ep2, sr  = librosa.load(audio_ep2)
signal_ep3, sr  = librosa.load(audio_ep3)

f0s_ep1 = librosa.yin(signal_ep1, librosa.note_to_hz('C2'), librosa.note_to_hz('C7')) 
f0s_ep2 = librosa.yin(signal_ep2, librosa.note_to_hz('C2'), librosa.note_to_hz('C7')) 
f0s_ep3 = librosa.yin(signal_ep3, librosa.note_to_hz('C2'), librosa.note_to_hz('C7')) 

np.save(time_audio_loc + 'yin_f0s_ep1.npy', f0s_ep1)
np.save(time_audio_loc + 'yin_f0s_ep2.npy', f0s_ep2)
np.save(time_audio_loc + 'yin_f0s_ep3.npy', f0s_ep3)

**B) Time domain Video**

Extraction of previously named Time Domain Video Features:
- Edge Information (Contour Information)
- Edge Orientation Histogram
- HUE Information (Dominant colour)
- HUE Information - Color Histograms :) <br>

In [7]:
# Image Array Information
gd_loc_ep1 = "/content/drive/MyDrive/data/processed_data/episodes/ep1/"
gd_loc_ep2 = "/content/drive/MyDrive/data/processed_data/episodes/ep2/"
gd_loc_ep3 = "/content/drive/MyDrive/data/processed_data/episodes/ep3/"

ep_paths = []

ep_paths.append(gd_loc_ep1 + "frame_array_ep1_1.npy")
ep_paths.append(gd_loc_ep1 + "frame_array_ep1_2.npy")
ep_paths.append(gd_loc_ep1 + "frame_array_ep1_3.npy")
ep_paths.append(gd_loc_ep1 + "frame_array_ep1_4.npy")

ep_paths.append(gd_loc_ep2 + "frame_array_ep2_1.npy")
ep_paths.append(gd_loc_ep2 + "frame_array_ep2_2.npy")
ep_paths.append(gd_loc_ep2 + "frame_array_ep2_3.npy")
ep_paths.append(gd_loc_ep2 + "frame_array_ep2_4.npy")

ep_paths.append(gd_loc_ep3 + "frame_array_ep3_1.npy")
ep_paths.append(gd_loc_ep3 + "frame_array_ep3_2.npy")
ep_paths.append(gd_loc_ep3 + "frame_array_ep3_3.npy")
ep_paths.append(gd_loc_ep3 + "frame_array_ep3_4.npy")

# Saving Calculated Features to save time in the future
video_feature_save_loc = "/content/drive/MyDrive/data/processed_data/time_video/"

In [ ]:
# Edge Information, Number of Edges, Number of Objects in Image
# NOTE: The cv2 Canny method uses a Sobel Kernel for Edge Computation
feature_vector_objects = []
feature_vector_edge_histograms = []
feature_vector_contour_areas = []
feature_vector_contour_centroids = []

for ep_path in ep_paths:
  ep_batch = np.load(ep_path)

  edge_batch = []
  for image in ep_batch:

    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image_with_edges = cv2.Canny(gray , 100, 200)
    
    contours, hierarchy= cv2.findContours(image_with_edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    number_of_objects_in_image = len(contours)
    feature_vector_objects.append(number_of_objects_in_image)
    
    cnt_areas = []
    cnt_centroids = [] 
    for cnt in contours:
      M = cv2.moments(cnt)
      if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
      else:
      # set values as what you need in the situation
        cX, cY = 0, 0

      area = cv2.contourArea(cnt)
      cnt_areas.append(area)
      cnt_centroids.append((cx,cy))

    feature_vector_contour_areas.append(cnt_areas)
    feature_vector_contour_centroids.append(cnt_centroids)

    hist = cv2.calcHist([image_with_edges],[0],None,[16],[0,256])
    feature_vector_edge_histograms.append(hist)

np.save(video_feature_save_loc + 'countours_feature_vector.npy', feature_vector_objects)
np.save(video_feature_save_loc + 'countour_areas_feature_vector.npy', feature_vector_contour_areas)
np.save(video_feature_save_loc + 'countour_centroids_feature_vector.npy', feature_vector_contour_centroids)
np.save(video_feature_save_loc + 'countour_histograms_feature_vector.npy', feature_vector_edge_histograms)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [13]:
# Edge Orientation Histogram

# 1. Clalculate gradient
# 2. Calculate histogram

feature_vector_eoh = [] 
feature_vector_gradient = []

for ep_path in ep_paths:
  ep_batch = np.load(ep_path)

  edge_batch = []
  for image in ep_batch:
    # First we do the Sobel Gradient
    sobelx = cv2.Sobel(image,cv2.CV_64F,1,0,ksize=5)
    sobely = cv2.Sobel(image,cv2.CV_64F,0,1,ksize=5)
    # And get combined gradient
    grad  = np.sqrt(sobelx**2 + sobely**2)
    abs_grad = np.absolute(grad)
    grad_8u = np.uint8(abs_grad)
    # Now we make a nice little histogram
    hist = cv2.calcHist([grad_8u],[0],None,[256],[0,256])
    
    feature_vector_eoh.append(hist)
    #feature_vector_gradient.append(grad_8u)

np.save(video_feature_save_loc + 'eoh_feature_vector.npy', feature_vector_eoh)
#np.save(video_feature_save_loc + 'gradient_feature_vector.npy', feature_vector_gradient)

In [17]:
 # HUE Information
 # Dominant Color
feature_vector_dominant_color = []

for ep_path in ep_paths:
  ep_batch = np.load(ep_path)

  for image in ep_batch:
    
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv_img],[0],None,[256],[0,256])

    #Convert histogram to simple list
    hist = [val[0] for val in hist]; 
    #Generate a list of indices
    indices = list(range(0, 256));
    #Descending sort-by-key with histogram value as key
    s = [(x,y) for y,x in sorted(zip(hist,indices), reverse=True)]
    #Index of highest peak in histogram
    index_of_max = s[0][0];
    #Index of second highest peak in histogram
    index_of_max2 = s[1][0];

    feature_vector_dominant_color.append((hist[index_of_max], hist[index_of_max2]))
    
np.save(video_feature_save_loc + 'feature_vector_dominant_color.npy', feature_vector_dominant_color) 

In [19]:
# HUE Information
# Color Histograms - Just to keep the proffesor interested :)
feature_vector_color_histogram = []

for ep_path in ep_paths:
  ep_batch = np.load(ep_path)

  for image in ep_batch:
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv_img],[0],None,[256],[0,256])

    feature_vector_color_histogram.append(hist)

np.save(video_feature_save_loc + 'feature_vector_color_histogram.npy', feature_vector_color_histogram) 

C) **Classification** (Experimentation)

In this section we will use the calculated Audio and Visual features for classification.
<br>
I have decided to use the K-Nearest-Neighbours Classifier, as it seems to be very popular for these kinds of tasks

In [7]:
# Feature Vector Locations

audio_time_save_loc = "/content/drive/MyDrive/data/processed_data/time_audio/"
video_time_save_loc = "/content/drive/MyDrive/data/processed_data/time_video/"

In [52]:
# Detecting Audio for Stadtler and Waldorf

y_train = statler_waldorf_audio_train
y_test = statler_waldorf_audio_test

# RMS

# Loudness LOAD-UP
rms_ep1_all = np.array(np.load(audio_time_save_loc + 'rms_ep1.npy'))
rms_ep2_all = np.array(np.load(audio_time_save_loc + 'rms_ep2.npy'))
rms_ep3_all = np.array(np.load(audio_time_save_loc + 'rms_ep3.npy'))

rms_train = np.hstack((rms_ep1_all, rms_ep2_all)) 

rms_test = np.asarray(rms_ep3_all)



model_audio_rms = KNeighborsClassifier(n_neighbors = 5)
model_audio_rms.fit(rms_train, y_train)

## TEST

outputs_rms = model_audio.predict(rms_test)

# Calculate Precision, Recall and F-measure

tp = 0
fp = 0
fn = 0

for (output, label) in zip(outputs_rms, y_test):
  if(output == 1 and label == 1):
    tp += 1
  if(output == 1 and label == 0):
    fp += 1
  if(output == 0 and label == 0):
    fn += 1  


ValueError: ignored

In [51]:
# Detecting Audio for Stadtler and Waldorf
y_train = statler_waldorf_audio_train
y_test = statler_waldorf_audio_test

# Fundamental Frequencies
# Fundamental ZCR LOAD-UP
zcr_ep1 = np.load(audio_time_save_loc + 'zcr_ep1.npy')
zcr_ep2 = np.load(audio_time_save_loc + 'zcr_ep2.npy')
zcr_ep3 = np.load(audio_time_save_loc + 'zcr_ep3.npy')

zcr_ep1 = np.array(zcr_ep1)
zcr_ep2 = np.array(zcr_ep2)
zcr_ep3 = np.array(zcr_ep3)

zcr_train = np.hstack((zcr_ep1, zcr_ep2))
zcr_test = zcr_ep3

print(zcr_train.shape)

# Fundamental YIN LOAD-UP
f0s_ep1 = np.load(time_audio_loc + 'yin_f0s_ep1.npy')
f0s_ep2 = np.load(time_audio_loc + 'yin_f0s_ep2.npy')
f0s_ep3 = np.load(time_audio_loc + 'yin_f0s_ep3.npy')

f0s_ep1 = np.array(f0s_ep1)
f0s_ep2 = np.array(f0s_ep2)
f0s_ep3 = np.array(f0s_ep3)

f0s_train = np.hstack((f0s_ep1, f0s_ep2))
f0s_test = f0s_ep3

feature_vector_final_train = np.vstack((zcr_train,f0s_train))

feature_vector_final_test = np.vstack((zcr_test,f0s_test))


model_audio_ff = KNeighborsClassifier(n_neighbors = 5)
model_audio_ff.fit(feature_vector_final_train, y_train)

## TEST

outputs_ff = model_audio.predict(feature_vector_final_test)

# Calculate Precision, Recall and F-measure

tp = 0
fp = 0
fn = 0

for (output, label) in zip(outputs_ff, y_test):
  if(output == 1 and label == 1):
    tp += 1
  if(output == 1 and label == 0):
    fp += 1
  if(output == 0 and label == 0):
    fn += 1  


(1, 133277)
(2, 133277)


ValueError: ignored

In [81]:
# Video for Kermit
# Dominant Color
y_train = kermit_train
y_test = kermit_test
train_size = len(y_train)

# Color Dominance LOAD-UP
feature_vector_dominant_color = np.load(video_time_save_loc + 'feature_vector_dominant_color.npy') 

feature_vector_train = feature_vector_dominant_color[range(0, train_size)][:]
feature_vector_test = feature_vector_dominant_color[range(train_size, len(feature_vector_dominant_color))][:]

model_cd = KNeighborsClassifier(n_neighbors = 3)
model_cd.fit(feature_vector_train, y_train)

## TEST

outputs = model_cd.predict(feature_vector_test)

# Calculate Precision, Recall and F-measure

tp = 0
fp = 0
fn = 0

count = 0
for (output, label) in zip(outputs, y_test):
  count += 1
  if(output == 1 and label == 1):
    tp += 1
  if(output == 1 and label == 0):
    fp += 1
  if(output == 0 and label == 0):
    fn += 1  

precision = tp/(tp + fp)
recall = tp/(tp+fn)
fmeasure = 2 * precision * recall / (precision + recall)

print(precision)
print(recall)
print(fmeasure)


(77387, 2)
0.40325670498084293
0.09576888080072793
0.1547794117647059


prec = 0.41214549938347717
recall = 0.05963425512934879
fmeasure = 0.10419264339152119

Conclusion: Dominant Color Works Horribly

In [87]:
# Video for Kermit
#feature_vector_color_histogram

y_train = kermit_train
y_test = kermit_test
train_size = len(y_train)

# Color Histogram LOAD-UP
feature_vector_color_histogram = np.load(video_time_save_loc + 'feature_vector_color_histogram.npy') 

feature_vector_train = feature_vector_color_histogram[range(0, train_size)][:]
feature_vector_test = feature_vector_color_histogram[range(train_size, len(feature_vector_color_histogram))][:]

print(feature_vector_train.shape)

nsamples, nx, ny = feature_vector_train.shape
d2_train_dataset = feature_vector_train.reshape((nsamples,nx*ny))

model_cd = KNeighborsClassifier(n_neighbors = 3)
model_cd.fit(d2_train_dataset, y_train)

## TEST

nsamples, nx, ny = feature_vector_test.shape
d2_test_dataset = feature_vector_test.reshape((nsamples,nx*ny))

outputs = model_cd.predict(d2_test_dataset)

# Calculate Precision, Recall and F-measure

tp = 0
fp = 0
fn = 0

count = 0
for (output, label) in zip(outputs, y_test):
  count += 1
  if(output == 1 and label == 1):
    tp += 1
  if(output == 1 and label == 0):
    fp += 1
  if(output == 0 and label == 0):
    fn += 1  

precision = tp/(tp + fp)
recall = tp/(tp+fn)
fmeasure = 2 * precision * recall / (precision + recall)

print(precision)
print(recall)
print(fmeasure)


(77387, 256, 1)
0.45979614949037373
0.07152294547696644
0.12378992301242472


Precision:0.45979614949037373 Recall:0.07152294547696644 F-measure:0.12378992301242472
Color histogram, also horrible with KNN

In [ ]:
# Video for Kermit
# Edge/Contour Information

y_train = kermit_train
y_test = kermit_test
train_size = len(y_train)

# Edge Information LOAD-UP
feature_vector_objects = np.array(np.load(video_time_save_loc + 'countours_feature_vector.npy'))
feature_vector_countour_areas = np.array(np.load(video_time_save_loc + 'countour_areas_feature_vector.npy', allow_pickle = True))
feature_vector_countour_centroids = np.array(np.load(video_time_save_loc + 'countour_centroids_feature_vector.npy', allow_pickle = True))

feature_vector_final = np.dstack((feature_vector_objects, feature_vector_countour_areas))
#feature_vector_final = feature_vector_final.transpose(1,2,0)


print(feature_vector_final.shape)
del feature_vector_objects
del feature_vector_countour_areas
del feature_vector_countour_centroids

feature_vector_train = feature_vector_final[range(0, train_size)][:]
feature_vector_test = feature_vector_final[range(train_size, len(feature_vector_final))][:]

nsamples, nx, ny = feature_vector_train.shape
d2_train_dataset = feature_vector_train.reshape((nsamples,nx*ny))

model_cd = KNeighborsClassifier(n_neighbors = 3)
model_cd.fit(d2_train_dataset, y_train)

## TEST


outputs = model_cd.predict(feature_vector_test)

# Calculate Precision, Recall and F-measure

tp = 0
fp = 0
fn = 0

count = 0
for (output, label) in zip(outputs, y_test):
  count += 1
  if(output == 1 and label == 1):
    tp += 1
  if(output == 1 and label == 0):
    fp += 1
  if(output == 0 and label == 0):
    fn += 1  

precision = tp/(tp + fp)
recall = tp/(tp+fn)
fmeasure = 2 * precision * recall / (precision + recall)

print(precision)
print(recall)
print(fmeasure)


(3, 115888)


In [ ]:
# Video for Kermit
# Edge Orientation Histograms

y_train = kermit_train
y_test = kermit_test
train_size = len(y_train)

# Edge Orientation HISTOGRAM LOAD-UP
feature_vector_eoh = np.load(video_time_save_loc + 'eoh_feature_vector.npy')

feature_vector_train = feature_vector_eoh[range(0, train_size)][:]
feature_vector_test = feature_vector_eoh[range(train_size, len(feature_vector_eoh))][:]

nsamples, nx, ny = feature_vector_train.shape
d2_train_dataset = feature_vector_train.reshape((nsamples,nx*ny))

model_cd = KNeighborsClassifier(n_neighbors = 3)
model_cd.fit(d2_train_dataset, y_train)

## TEST


outputs = model_cd.predict(feature_vector_test)

# Calculate Precision, Recall and F-measure

tp = 0
fp = 0
fn = 0

count = 0
for (output, label) in zip(outputs, y_test):
  count += 1
  if(output == 1 and label == 1):
    tp += 1
  if(output == 1 and label == 0):
    fp += 1
  if(output == 0 and label == 0):
    fn += 1  

precision = tp/(tp + fp)
recall = tp/(tp+fn)
fmeasure = 2 * precision * recall / (precision + recall)

print(precision)
print(recall)
print(fmeasure)

In [ ]:
# Video + Audio for Kermit
y_train = kermit_train
y_test = kermit_test

neigh = KNeighborsClassifier(n_neighbors = 5)
neigh.fit(X, y)

  if(output == 1 and label == 1):
    tp += 1
  if(output == 1 and label == 0):
    fp += 1
  if(output == 0 and label == 0):
    fn += 1  

D) Evaluation and Visualisation

E) Comment

# Similarity Modelling 2

The features relevant for Similarity Modelling 2: <br>


**Audio:**

Frequency domain features:
- The mel frequency cepstral coefficients (MFCCs)
- Spectral peaks

**Video:**

Frequency domain features:
- Luminance + Discrete Cosine Transform <br>

Local methods:
- SIFT - Scale Invariant Feature Transformation
  (Bag of Visual Features Variant)
- HOG - Histogram of Oriented Gradients
- Shi-Tomasi Corner Detection


**Frequency Domain Feature Understanding**

Extracting the audio features.

I put in some effort to understand multiple audio signal processing methods.
Including Spectrogram creations, DFTs, Beat Detections, OnSet Detections, the YIN method 

**A) Audio Features (Frequency domain)**

Most of the frequency domain calculations refer to a spectrogram
Some time to explain what the spectrorgram is:

- We can calculate a Fourier Transform for a signal of a certain size (e.g. 2048 frames)
- If we do this with a sliding window over the whole signal - we get a spectrogram

Extraction of previously named Frequency Domain Audio Features: <br>
- The mel frequency cepstral coefficients (MFCCs)
- Spectral peaks <br>

In [8]:
freq_audio_loc = "/content/drive/MyDrive/data/processed_data/frequency_audio/"

In [9]:
# MFCCs 
# Calculating the mel log spectrogram, and then doing the factor analysis
signal_ep1, sr  = librosa.load(audio_ep1)
signal_ep2, sr  = librosa.load(audio_ep2)
signal_ep3, sr  = librosa.load(audio_ep3)

mfcc_ep1 = librosa.feature.mfcc(y = signal_ep1, sr=sr)
mfcc_ep2 = librosa.feature.mfcc(y = signal_ep2, sr=sr)
mfcc_ep3 = librosa.feature.mfcc(y = signal_ep3, sr=sr)

np.save(freq_audio_loc + 'mfcc_ep1.npy', mfcc_ep1)
np.save(freq_audio_loc + 'mfcc_ep2.npy', mfcc_ep2)
np.save(freq_audio_loc + 'mfcc_ep3.npy', mfcc_ep3)

print(len(mfcc_ep1))
del mfcc_ep1
del mfcc_ep2
del mfcc_ep3

20


In [ ]:
# Spectral peaks method
# First we calculate spectrograms for each signal
# Then we calculate energy contrasts
signal_ep1, sr  = librosa.load(audio_ep1)
signal_ep2, sr  = librosa.load(audio_ep2)
signal_ep3, sr  = librosa.load(audio_ep3)

spectrogram_ep1 = np.abs(librosa.stft(signal_ep1))
contrast_ep1 = librosa.feature.spectral_contrast(S=spectrogram_ep1, sr=sr)
np.save(freq_audio_loc + 'contrast_ep1.npy', contrast_ep1)
print(len(contrast_ep1))
del contrast_ep1
del spectrogram_ep1

spectrogram_ep2 = np.abs(librosa.stft(signal_ep2))
contrast_ep2 = librosa.feature.spectral_contrast(S=spectrogram_ep2, sr=sr)
np.save(freq_audio_loc + 'contrast_ep2.npy', contrast_ep2)
del contrast_ep2
del spectrogram_ep2

spectrogram_ep3 = np.abs(librosa.stft(signal_ep3))
contrast_ep3 = librosa.feature.spectral_contrast(S=spectrogram_ep3, sr=sr)
np.save(freq_audio_loc + 'contrast_ep3.npy', contrast_ep3)
del contrast_ep3
del spectrogram_ep3

# From energy contrasts we can find the maximum values

**B) Video Features (Frequency Domain)**

Extraction of previously named Frequency Domain Video Features: <br>

- Luminance + Discrete Cosine Transform

In [9]:
video_freq_save_loc = "/content/drive/MyDrive/data/processed_data/frequency_video/"

In [18]:
# Luminance + Discrete Cosine Transform
dct_feature_vector = []
comp = 0

for ep_path in ep_paths:
  ep_batch = np.load(ep_path)

  if(comp >= 20):
    break

  edge_batch = []
  for image in ep_batch:
    comp += 1
    # calculate the Discrete Cosine Transform
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dct_img = scipy.fftpack.dct(img)

    if(comp == 1):
      print(dct_img.shape)

    if(comp == 20):
      print(dct_img.shape)
      break

    dct_feature_vector.append(dct[range(0,28)][range(0:36)])

np.save(video_freq_save_loc + 'feature_vector_dct.npy', dct_feature_vector)

(272, 360)
(272, 360)


**C) Video Features (Local methods)**

Local methods:
- SIFT - Scale Invariant Feature Transformation
  (Bag of Visual Features Variant)
- HOG - Histogram of Oriented Gradients
- Shi-Tomasi Corner Detection

In [25]:
video_feature_save_loc = "/content/drive/MyDrive/data/processed_data/local_video/"

In [27]:
# Histogram of Oriented Gradients (HOG)
# We have to rescale, otherwise HOG becomes far too large
vidcap = cv2.VideoCapture(path_ep1)
(success,image) = vidcap.read()
scale_percent = 30 # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)

feature_vector_hog = []

for ep_path in ep_paths:
  ep_batch = np.load(ep_path)

 

  for image in ep_batch:
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    gray = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)
    H = feature.hog(gray, orientations=9, pixels_per_cell=(10, 10),
	    cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1")
    feature_vector_hog.append(H)

np.save(video_feature_save_loc + 'feature_vector_hog.npy', feature_vector_hog)

KeyboardInterrupt: ignored

In [13]:
# SIFT
# We use cv2 for SIFT
# The way it works:
# 1. Finds IPs - interest points (Exist on multiple coarsness levels)
# 2. Save IP neighbourhoods: big neighbourhood -> more important point
# 3. Calculates Gradient Histrogram
# We then use SIFT features to create histograms
n_samples = 20
total_SIFT_features = []
vocab_size = 200
feats = []

sift = cv2.xfeatures2d.SIFT_create()

for ep_path in ep_paths:
  ep_batch = np.load(ep_path)

  for image in ep_batch:
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(gray,None)

    if(des is None):
      for i in range(0, n_samples):
        total_SIFT_features.append(np.zeros(128))
    else:
      des_samples = des[np.random.randint(des.shape[0], size = n_samples)]

      for i in range(0, n_samples):
        total_SIFT_features.append(des_samples[i][:])

#np.save(video_feature_save_loc + 'SIFT_totals.npy', total_SIFT_features)
#total_SIFT_features = np.load(video_feature_save_loc + 'SIFT_totals.npy')
vocab = scipy.cluster.vq.kmeans(total_SIFT_features, vocab_size)

del total_SIFT_features

np.save(video_feature_save_loc + 'SIFT_vocab.npy', vocab)

# Calculating SIFT feature Histograms
for ep_path in ep_paths:
  ep_batch = np.load(ep_path)

  for image in ep_batch:
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(gray,None)

    dist = scipy.spatial.distance.cdist(des, vocab, "euclidean")

    bin_assignment = np.argmin(dist, axis = 1)

    image_features = np.zeros(200)

    for bin in bin_assignment:
      image_feats[id] += 1

    feats.append(image_feats)

feats = np.asarray(feats)
feats_norm_div = np.linalg.norm(feats, axis = 1)

for i in range(0, feats.shape[0]):
  feats[i] = feats[i] / feats_norm_div[i]  

np.save(video_feature_save_loc + 'feature_vector_SIFT_bag.npy', feats)


KeyboardInterrupt: ignored

In [21]:
# Shi-Tomasi Corner Detector

feature_vector_shi_tomasi = []

for ep_path in ep_paths:
  ep_batch = np.load(ep_path)

  for image in ep_batch:
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    corners = cv2.goodFeaturesToTrack(gray,25,0.01,10)
    
    if corners is not None:
      for i in corners:
        x,y = i.ravel()
        image = cv2.circle(image,(x,y),3,255,-1)
    
    hist = cv2.calcHist([image],[0],None,[256],[0,256])

    feature_vector_shi_tomasi.append(hist)

np.save(video_feature_save_loc + 'feature_vector_shi_tomasi.npy', feature_vector_shi_tomasi)

C) **Classification** (and Experimentation)

In [20]:
# Feature Vector Locations

audio_time_save_loc = "/content/drive/MyDrive/data/processed_data/time_audio/"
video_time_save_loc = "/content/drive/MyDrive/data/processed_data/time_video/"
audio_freq_save_loc = "/content/drive/MyDrive/data/processed_data/frequency_audio/"
video_local_save_loc = "/content/drive/MyDrive/data/processed_data/local_video/"
video_freq_save_loc = "/content/drive/MyDrive/data/processed_data/frequency_video/"

In [ ]:
# MFCC LOAD-UP
mfcc_ep1 = np.load(audio_freq_save_loc + 'mfcc_ep1.npy')
mfcc_ep2 = np.load(audio_freq_save_loc + 'mfcc_ep2.npy')
mfcc_ep3 = np.load(audio_freq_save_loc + 'mfcc_ep3.npy')

In [ ]:
# SPECTRAL CONTRAS LOAD-UP
contrast_ep1 = np.load(audio_freq_save_loc + 'contrast_ep1.npy')
contrast_ep2 = np.load(audio_freq_save_loc + 'contrast_ep2.npy')
contrast_ep3 = np.load(audio_freq_save_loc + 'contrast_ep3.npy')

In [ ]:
# Luminance + DCT LOAD-UP
dct_feature_vector = np.load(video_freq_save_loc + 'feature_vector_dct.npy')

In [ ]:
# HOG LOAD-UP
feature_vector_hog = np.save(video_local_save_loc + 'feature_vector_hog.npy')

In [ ]:
# Shi Tomasi LOAD-UP
feature_vector_shi_tomasi = np.save(video_local_save_loc + 'feature_vector_shi_tomasi.npy')

In [ ]:
# SIFT Bag LOAD-UP
feature_vector_SIFT_bag = np.save(video_local_save_loc + 'feature_vector_SIFT_bag.npy')

In [ ]:
# Audio Feature Classification for Stadter and Waldorf
# MFCC Classifier
y_train = statler_waldorf_audio_train
y_test = statler_waldorf_audio_test

mfcc_ep1 
mfcc_ep2 
mfcc_ep3

clf = svm.SVC()

clf.fit(X, y_test)


predictions = clf.predict(X)

tp = 0
fp = 0
fn = 0

for (output, label) in zip(predictions, y_test):
  if(output == 1 and label == 1):
    tp += 1
  if(output == 1 and label == 0):
    fp += 1
  if(output == 0 and label == 0):
    fn += 1  

In [ ]:
# Audio Feature Classification for Stadter and Waldorf
# Spectral Peaks

contrast_ep1 
contrast_ep2 
contrast_ep3

clf = svm.SVC()

clf.fit(X, y_test)


predictions = clf.predict(X)

tp = 0
fp = 0
fn = 0

for (output, label) in zip(predictions, y_test):
  if(output == 1 and label == 1):
    tp += 1
  if(output == 1 and label == 0):
    fp += 1
  if(output == 0 and label == 0):
    fn += 1  

In [24]:
# Video for Kermit
# HOG Classification

y_train = kermit_train
y_test = kermit_test
train_size = len(y_train)

# HOG LOAD-UP
feature_vector_hog = np.load(video_local_save_loc + 'feature_vector_hog.npy')

print(feature_vector_hog.shape)

feature_vector_train = feature_vector_hog[range(0, train_size)][:]
feature_vector_test = feature_vector_hog[range(train_size, len(feature_vector_hog))][:]

print(feature_vector_train.shape)

nsamples, nx, ny = feature_vector_train.shape
d2_train_dataset = feature_vector_train.reshape((nsamples,nx*ny))

clf = svm.SVC()

clf.fit(feature_vector_train, y_train)

#TEST

nsamples, nx, ny = feature_vector_test.shape
d2_test_dataset = feature_vector_test.reshape((nsamples,nx*ny))

outputs = clf.predict(feature_vector_test)

# Calculate Precision, Recall and F-measure

tp = 0
fp = 0
fn = 0

count = 0
for (output, label) in zip(outputs, y_test):
  count += 1
  if(output == 1 and label == 1):
    tp += 1
  if(output == 1 and label == 0):
    fp += 1
  if(output == 0 and label == 0):
    fn += 1  

precision = tp/(tp + fp)
recall = tp/(tp+fn)
fmeasure = 2 * precision * recall / (precision + recall)

print(precision)
print(recall)
print(fmeasure)



(8497, 10800)


IndexError: ignored

In [ ]:
# Video for Kermit
# Shi-Tomasi Corner Detection
y_train = kermit_train
y_test = kermit_test
train_size = len(y_train)

feature_vector_shi_tomasi = np.load(video_local_save_loc + 'feature_vector_shi_tomasi.npy')

print(feature_vector_shi_tomasi.shape)

feature_vector_train = feature_vector_shi_tomasi[range(0, train_size)][:]
feature_vector_test = feature_vector_shi_tomasi[range(train_size, len(feature_vector_shi_tomasi))][:]

print(feature_vector_train.shape)

nsamples, nx, ny = feature_vector_train.shape
d2_train_dataset = feature_vector_train.reshape((nsamples,nx*ny))

clf = svm.SVC()

clf.fit(d2_train_dataset, y_train)

#TEST

nsamples, nx, ny = feature_vector_test.shape
d2_test_dataset = feature_vector_test.reshape((nsamples,nx*ny))

outputs = clf.predict(d2_test_dataset)

# Calculate Precision, Recall and F-measure

tp = 0
fp = 0
fn = 0

count = 0
for (output, label) in zip(outputs, y_test):
  count += 1
  if(output == 1 and label == 1):
    tp += 1
  if(output == 1 and label == 0):
    fp += 1
  if(output == 0 and label == 0):
    fn += 1  

precision = tp/(tp + fp)
recall = tp/(tp+fn)
fmeasure = 2 * precision * recall / (precision + recall)

print(precision)
print(recall)
print(fmeasure)



(115888, 256, 1)
(77387, 256, 1)


In [ ]:
# Video for Kermit
# DCT + Luminance
y_train = kermit_train
y_test = kermit_test
train_len = len(y_train)

print(dct_feature_vector.shape)

clf = svm.SVC()

clf.fit(X, y_test)

predictions = clf.predict(X)

tp = 0
fp = 0
fn = 0

for (output, label) in zip(predictions, y_test):
  if(output == 1 and label == 1):
    tp += 1
  if(output == 1 and label == 0):
    fp += 1
  if(output == 0 and label == 0):
    fn += 1  

D) Evaluation and Visualisation

E) Comment

SIFT:
- In the end the Bag of Visual Features Algorithm using SIFT proved
to take too long to compute for a dataset of this size, however
the key ideas were learned and this Feature Descriptor can prove to
be valuable on small Image datasets

